# 3.2 Read write parquet files

In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import os

In [2]:
local=True
if local:
    spark=SparkSession.builder.master("local[4]") \
                  .appName("ReadWriteParquet").getOrCreate()
else:
    spark=SparkSession.builder \
                      .master("k8s://https://kubernetes.default.svc:443") \
                      .appName("ReadWriteParquet") \
                      .config("spark.kubernetes.container.image","inseefrlab/jupyter-datascience:master") \
                      .config("spark.kubernetes.authenticate.driver.serviceAccountName",os.environ['KUBERNETES_SERVICE_ACCOUNT']) \
                      .config("spark.kubernetes.namespace", os.environ['KUBERNETES_NAMESPACE']) \
                      .config("spark.executor.instances", "4") \
                      .config("spark.executor.memory","8g") \
                      .config("spark.kubernetes.driver.pod.name", os.environ["POD_NAME"]) \
                      .config('spark.jars.packages','org.postgresql:postgresql:42.2.24') \
                      .getOrCreate()

22/02/12 02:22:32 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.184.146 instead (on interface ens33)
22/02/12 02:22:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/02/12 02:22:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
file_path="data/adult.snappy.parquet"
df=spark.read.parquet(file_path)

In [5]:
df.show(5)

+---+----------------+------+---------+-------------+------------------+-----------------+-------------+-----+------+------------+------------+--------------+--------------+------+
|age|       workclass|fnlwgt|education|education-num|    marital-status|       occupation| relationship| race|   sex|capital-gain|capital-loss|hours-per-week|native-country|income|
+---+----------------+------+---------+-------------+------------------+-----------------+-------------+-----+------+------------+------------+--------------+--------------+------+
| 39|       State-gov| 77516|Bachelors|           13|     Never-married|     Adm-clerical|Not-in-family|White|  Male|        2174|           0|            40| United-States| <=50K|
| 50|Self-emp-not-inc| 83311|Bachelors|           13|Married-civ-spouse|  Exec-managerial|      Husband|White|  Male|           0|           0|            13| United-States| <=50K|
| 38|         Private|215646|  HS-grad|            9|          Divorced|Handlers-cleaners|Not-i

In [6]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: integer (nullable = true)
 |-- education: string (nullable = true)
 |-- education-num: integer (nullable = true)
 |-- marital-status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital-gain: integer (nullable = true)
 |-- capital-loss: integer (nullable = true)
 |-- hours-per-week: integer (nullable = true)
 |-- native-country: string (nullable = true)
 |-- income: string (nullable = true)



In [8]:
row_numbers=df.count()
print(f"row numbers: {row_numbers}")

row numbers: 32561


# Write parquet with specific partition number

In [10]:
partition_number=df.rdd.getNumPartitions()
print(f"Data partition number: {partition_number}")

Data partition number: 1


In [27]:
output_path="data/out_test"
df.write.mode("overwrite").parquet(output_path)

In [28]:
!ls {output_path}

part-00000-27340c38-d52a-4e2d-a4a7-7a356624f801-c000.snappy.parquet  _SUCCESS


You can notice, the above dataframe write to only one parquet file. That's because the partition number of the dataframe is 1. Now let's try to repartition the dataframe and write to parquet file again.

In [20]:
# check the execution plan
df.repartition(4,col("age")).explain()

== Physical Plan ==
Exchange hashpartitioning(age#0, 4), REPARTITION_WITH_NUM, [id=#151]
+- *(1) ColumnarToRow
   +- FileScan parquet [age#0,workclass#1,fnlwgt#2,education#3,education-num#4,marital-status#5,occupation#6,relationship#7,race#8,sex#9,capital-gain#10,capital-loss#11,hours-per-week#12,native-country#13,income#14] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex[file:/home/pliu/git/PySparkCommonFunc/notebooks/pysparkbasics/L03_ReadFromVario..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<age:int,workclass:string,fnlwgt:int,education:string,education-num:int,marital-status:stri...




In [21]:
# Note all dataframe are immutable, you need to create a new dataframe to save the modified dataframe
df1=df.repartition(4,col("age"))

In [22]:
# now you should see the partition number is 4
partition_number1=df1.rdd.getNumPartitions()
print(f"Data partition number: {partition_number1}")

Data partition number: 4


In [24]:
df1.write.mode("overwrite").parquet(output_path)

In [26]:
!ls {output_path}

part-00000-d8a1f2a5-4b5c-4f4c-8c04-d7972e3f9fcd-c000.snappy.parquet
part-00001-d8a1f2a5-4b5c-4f4c-8c04-d7972e3f9fcd-c000.snappy.parquet
part-00002-d8a1f2a5-4b5c-4f4c-8c04-d7972e3f9fcd-c000.snappy.parquet
part-00003-d8a1f2a5-4b5c-4f4c-8c04-d7972e3f9fcd-c000.snappy.parquet
_SUCCESS


Now, you can notice we have 4 parquet file instead of 1. Because we changed the partition number of the dataframe

# Choose compression algo
You can notice, by default spark write parquet file with snappy compression.
